In [134]:
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

DataPath         =   './DataSet/dsp.stackexchange.com/CSV/'
PostsData        =   pd.read_csv(DataPath+'Posts.csv')

In [135]:
PostsData.head(2)

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount
0,1,1,20.0,2011-08-16T19:13:36.950,14,690.0,<p>Are there ways to increase computational pe...,6.0,1473.0,2012-06-06T13:43:10.863,2013-01-27T16:11:56.467,How do I improve LMS / NLMS filter performance?,<adaptive-filters><performance><least-squares>,1.0,0,1.0
1,5,1,27.0,2011-08-16T20:01:31.747,13,1951.0,<p>When developing products I do all of my alg...,17.0,17.0,2011-08-16T20:40:38.407,2011-09-07T14:32:39.710,How to decide what platform is best to impleme...,<arm><dsp-core><hardware>,3.0,8,7.0


In [136]:
Questions = PostsData[PostsData['PostTypeId']==1]
Answers   = PostsData[PostsData['PostTypeId']==2]

In [137]:
TargetQs =  Questions.dropna(subset = ['AcceptedAnswerId'])

In [138]:
TargetQs = TargetQs[['Id','AcceptedAnswerId','CreationDate', 'Title','Tags','Body']]

In [139]:
TargetQs['BLen']=TargetQs['Body'].apply(len)

In [140]:
def TagCounter(row):
    Tags = re.split('<|>',row)
    Tags = [Tag for Tag in Tags if len(Tag)>1]
    return len(Tags)

In [141]:
TargetQs['NTags']=TargetQs['Tags'].apply(TagCounter)

In [142]:
TargetQs['TLen']=TargetQs['Title'].apply(len)

In [143]:
TargetQs.rename(columns={'CreationDate':'QCreationDate'}, inplace=True)

In [144]:
TargetQs.AcceptedAnswerId = TargetQs.AcceptedAnswerId.astype(int)

In [145]:
TargetQs.head(2)

,Id,AcceptedAnswerId,QCreationDate,Title,Tags,Body,BLen,NTags,TLen
0,1,20,2011-08-16T19:13:36.950,How do I improve LMS / NLMS filter performance?,<adaptive-filters><performance><least-squares>,<p>Are there ways to increase computational pe...,479,3,47
1,5,27,2011-08-16T20:01:31.747,How to decide what platform is best to impleme...,<arm><dsp-core><hardware>,<p>When developing products I do all of my alg...,475,3,79


In [146]:
AnswersT = Answers[['Id','CreationDate']]

In [147]:
AnswersT.rename(columns={'Id':'AcceptedAnswerId'}, inplace=True)
AnswersT.rename(columns={'CreationDate':'ACreationDate'}, inplace=True)

In [148]:
AnswersT.head()

,AcceptedAnswerId,ACreationDate
5,9,2011-08-16T20:45:01.113
6,10,2011-08-16T21:07:30.923
7,11,2011-08-16T21:10:57.830
8,12,2011-08-16T21:20:29.243
10,14,2011-08-16T22:18:06.980


In [149]:
resultT = pd.merge(TargetQs, AnswersT, on='AcceptedAnswerId')

In [150]:
resultT['QCreationDate'] = pd.to_datetime(resultT.QCreationDate)
resultT['ACreationDate'] = pd.to_datetime(resultT.ACreationDate)
resultT['Tdelta']        = resultT['ACreationDate']-resultT['QCreationDate']

In [151]:
def delta2Minutes(row):
   return row.total_seconds()/3600

In [152]:
resultT['ATime']=resultT['Tdelta'].apply(delta2Minutes)

In [153]:
resultT = resultT[resultT['ATime']<=96]

In [154]:
resultT.to_csv('./ProcessedDataSet/resutT.csv', index=False)

In [155]:
# import matplotlib.pyplot as plt
# %matplotlib inline  
# fig, ax = plt.subplots()

# im = ax.scatter(resultT.BLen, resultT.ATime,s=5)
# plt.xlabel('Length of question Body')
# plt.ylabel('Hours taken to get an accepted answer')
# fig.savefig('EDA/Question_Body_Length_Vs_Hours.png', dpi = 600)

# im = ax.scatter(resultT.TLen, resultT.ATime,s=5)
# plt.xlabel('Length of question Title')
# plt.ylabel('Hours taken to get an accepted answer')
# fig.savefig('EDA/Question_Title_Length_Vs_Hours.png', dpi = 600)
 
# im = ax.scatter(resultT.TLen, resultT.BLen,c = resultT.ATime, s=5)
# fig.colorbar(im, ax=ax)
# plt.xlabel('Length of question Title')
# plt.ylabel('Length of question Body')
# fig.savefig('EDA/Question_Length_Vs_Hours.png', dpi = 600)

# im = ax.hist(resultT.TLen)
# plt.xlabel('Length of question Title')
# plt.ylabel('Count')
# fig.savefig('EDA/Question_Title_Histogram.png', dpi = 600)

# im = ax.hist(resultT.BLen)
# plt.xlabel('Length of question Body')
# plt.ylabel('Count')
# fig.savefig('EDA/Question_Body_Histogram.png', dpi = 600)